In [4]:
import json_lines as jl
import nltk
import xmltodict
from ispras import texterra as isp
import pymorphy2 as pm2
import gensim
import time as tm
from lxml import etree
import sys

isp_api = isp.API('ba74236a7212a71054ae1408b30b1bdef771d35b')
#nltk.download()

tokenizer = nltk.TweetTokenizer()
morph = pm2.MorphAnalyzer()  

# Лемматизация с помощью Texterra

Читаю комментарии из файла по очереди, разбиваю каждый на предложения, лемматизирую с помощью Texterra, составляя при этом словарь FullVocab. Записи в Fullvocab хранятся в следующем виде:

__item__:
- __id__: id нормальной формы
- __normal_form__: нормальная форма
- __tags__: часть речи
- __word_forms__: список встретившихся словоформ
- __cnt__: сколько раз встречалось
- __origin__: в каком корпусе текстов слово встретилось (t - треннировочный, a - целевой, e - дополнительный [например, вешний словарь])

В lemmatized.txt записываются построчно лемматизированные предложения, но вместо слов в нём "id нормальной формы", чтобы для word2vec слова "стать" (<i>глагол</i>) и "стать" (<i>сущ</i>) не были одним и тем же словом.

In [24]:
class FullVocab:
    
    # Инициализация, либо пустой класс, либо из файла vocab.xml
    def __init__(self, from_file = False):
        self.id_counter = 0
        self.vocab = []
        if from_file:
            f = open('vocab.xml', 'rb')
            root = etree.XML(f.read())
            for item in root:
                self.add_item(item[3].text.split(' '), item[1].text, item[2].text, item[5].text, cnt = item[4].text, 
                              word_id = item[0].text)
            
        
    # Добавляем новое слово в словарь
    def add_item(self, words, normal_form, tags_normal, origin, cnt = 0, word_id = -1):
        
        # Дабы было можно и str и list в аргументы добавлять
        if type(words) is not list:
            words = [words]
            
        # Проверяем, есть ли нормальная форма слова в словаре. Если есть, то обновляем словоформы, если нет, то 
        # добавляем новую запись в словарь, присвивая уникальный id
        for item in self.vocab:
            if item.get('normal_form') == normal_form and item.get('tags') == tags_normal:
                item['word_forms'].extend(words)
                item['word_forms'] = sorted(set(item['word_forms']))
                item['cnt'] += 1
                item['origin'] = self.handle_origin(origin, item['origin'])
                return item['id']
                
        #Если такой записи ещё не было
        new_item = {'normal_form': normal_form, 'tags': tags_normal, 'word_forms': words}
        if normal_form not in new_item['word_forms']:
            new_item['word_forms'].append(normal_form)
        if int(cnt) > 0:
            new_item['cnt'] = int(cnt)
        else:
            new_item['cnt'] = 1
            
        new_item['origin'] = self.handle_origin(origin)
        
        return_id = 0
        if word_id == -1:         #если мы не уточняем, какой именно id у нового слова (когда просто добавляем новое слово)
            new_item['id'] = self.id_counter
            return_id = self.id_counter
            self.id_counter = self.id_counter + 1
        else:                     #если мы указали id для нового слова (когда читаем из файла сформированный словарь)
            return_id = int(word_id)
            new_item['id'] = return_id
            self.id_counter = return_id + 1
            
        self.vocab.append(new_item)
        return return_id
         
        
    def show_vocab(self):
        print(self.vocab)
        
        
    # Сохраняем копию данные класса в vocab.xml
    def build_xml(self):
        f = open('vocab.xml', 'wb')
        root = etree.Element('root')
        for item in self.vocab:
            child = etree.SubElement(root, 'w' + str(item['id']))
            etree.SubElement(child, 'id').text = str(item['id'])
            etree.SubElement(child, 'normal_form').text = item['normal_form']
            etree.SubElement(child, 'tags').text = item['tags']
            etree.SubElement(child, 'word_forms').text = ' '.join(item['word_forms'])
            etree.SubElement(child, 'cnt').text = str(item['cnt'])
            etree.SubElement(child, 'origin').text = item['origin']
            
        result = etree.tostring(root, encoding='unicode', method='xml', pretty_print=True)
        f.write(result.encode('utf8'))
        
        
    # Ищем нормальную форму или запись в словаре по id
    # Если return_val = "word", то возвращаем нормальную форму, если "item", то запись в словаре
    def word_by_id(self, word_id, return_val = "word"):
        item = next((item for item in self.vocab if item["id"] == word_id))
        if item is None:
            return -1
        elif return_val == 'word':
            return item['normal_form']
        elif return_val == 'item':
            return item
        
        
    def id_by_word(self, word):
        item = next((item for item in self.vocab if item['normal_form'] == word))
        return item['id']
    
    
    def handle_origin(self, new, old = 'xxx'):
        if len(new) == 1:
            if new == 't':
                return 't' + old[1:3]
            elif new == 'a':
                return old[0] + 'a' + old[2]
            elif new == 'e':
                return  old[0:2] + 'e'
            
        elif len(new) == 3:
            return new

In [25]:
lemmatized_txt = open('lemmatized.txt', 'w', encoding='utf8')

with open('learn_scrapy\pda.jl', 'rb') as f:
    
    vocab = FullVocab()
    num_comments = 30242
    counter = 0
    prev_counter = 0
    anchor_counter = 0
    checkpoint = tm.clock()
    dt = 100
    
    for item in jl.reader(f):
        sents = nltk.sent_tokenize(item['comment'])
        for sent in sents:
            toks = tokenizer.tokenize(sent)
            toks = [t.lower() for t in toks if t.isalnum()]
            sent = ' '.join(toks)
            
            try:
                lemma_annotate = isp_api.lemmatizationAnnotate(text = sent)
                pos_annotate = isp_api.posTaggingAnnotate(text = sent)
                
            except KeyboardInterrupt:
                print("Understandable, have a nice day")
                vocab.build_xml()
                lemmatized_txt.close()
                sys.exit()
                
            except:
                print("Sentence", end = " \"")
                print(sent, end = "\" ")
                print("in commentary #", end = "")
                print(counter, end = " ")
                print("not processed")
                print("\n")
                continue
                
            if lemma_annotate.get('annotations').get('lemma') is not None:
                lemmatized = []
                for i in range(len(lemma_annotate.get('annotations').get('lemma'))):
                    l = lemma_annotate.get('annotations').get('lemma')[i].get('value')
                    p = pos_annotate['annotations']['pos-token'][i]['value'].get('tag')
                    word_id = vocab.add_item(toks[i], l, p, 't')
                    lemmatized.append(str(word_id))
                
                lemmatized = ' '.join(lemmatized)
                lemmatized_txt.write(lemmatized + '\n')
        
        counter = counter + 1
        if(tm.clock() - checkpoint > dt):
            print("Progress: ", end = '')
            print(counter / num_comments * 100, end = '')
            print("%")
            print("Comments processed by turn: ", end= '')
            print(counter - prev_counter)
            print("Comments processed total: ", end= '')
            print(counter)
            print("Time left: ", end = '')
            speed = (counter - prev_counter) / dt
            print(speed * (num_comments - counter))
            print("Speed: ", end = '')
            print(speed, end = ' ')
            print("comments/sec")
            print("\n")
            prev_counter = counter
            if (counter - anchor_counter) / num_comments * 100 > 33:
                print("Sleeping for 20 minutes")
                tm.sleep(1200)
                anchor_counter = counter
            checkpoint = tm.clock()
            
    vocab.build_xml()
    print("Finished")
    
lemmatized_txt.close()

Understandable, have a nice day


SystemExit: 

C:\Users\Maxim\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Building model using gensim
So basically using word2vec model from gensim

In [28]:
new_vocab = FullVocab(from_file = True)
new_vocab.show_vocab()
w2v_model = gensim.models.Word2Vec(gensim.models.word2vec.LineSentence('lemmatized.txt'), min_count=10)

[{'normal_form': 'если', 'origin': 'txx', 'tags': 'CONJ', 'id': 0, 'word_forms': ['если'], 'cnt': 1}, {'normal_form': 'она', 'origin': 'txx', 'tags': 'S', 'id': 1, 'word_forms': ['ее', 'она'], 'cnt': 2}, {'normal_form': 'только', 'origin': 'txx', 'tags': 'PART', 'id': 2, 'word_forms': ['только'], 'cnt': 1}, {'normal_form': 'на', 'origin': 'txx', 'tags': 'PR', 'id': 3, 'word_forms': ['на'], 'cnt': 5}, {'normal_form': 'алроид', 'origin': 'txx', 'tags': 'S', 'id': 4, 'word_forms': ['алроиды', 'алроид'], 'cnt': 1}, {'normal_form': 'и', 'origin': 'txx', 'tags': 'CONJ', 'id': 5, 'word_forms': ['и'], 'cnt': 7}, {'normal_form': 'айос', 'origin': 'txx', 'tags': 'S', 'id': 6, 'word_forms': ['айос'], 'cnt': 1}, {'normal_form': 'выходить', 'origin': 'txx', 'tags': 'V', 'id': 7, 'word_forms': ['вышла', 'выходить'], 'cnt': 1}, {'normal_form': 'где', 'origin': 'txx', 'tags': 'ADV', 'id': 8, 'word_forms': ['где'], 'cnt': 2}, {'normal_form': 'быть', 'origin': 'txx', 'tags': 'V', 'id': 9, 'word_forms': 

RuntimeError: you must first build vocabulary before training the model

# Some testing
Everythin' below is my lazy draft, don't borther yourself lookig through

In [12]:
comments = ''
with open('learn_scrapy\pda.jl', 'rb') as f:
    for item in jl.reader(f):
        comments = comments + item['comment']       #создаём один string на все комменты
                                                    #скорее всего, будет логичнее для N-граммов каждое предложение
                                                    #запоминать в один string и хранить как list()

In [13]:
#тут токенизируем комментарии
#есть подозрение, что я создаю слишком много сущностей 

tknzr = nltk.TweetTokenizer()
print('Kek1')
tokens = tknzr.tokenize(comments)
print('Kek2')
text = nltk.Text(tokens)
print('Kek3')
sents = nltk.sent_tokenize(comments)                         #строить n-граммную модель, возможно, буду 
print('Kek4')                                                             #анализируя предложения по-отдельности
words = sorted([w.lower() for w in tokens if w.isalpha()])   #впоследствии буду отсекать наименее наиболее частотные слова

Kek1
Kek2
Kek3
Kek4


In [20]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

print(lexical_diversity(text))

0.10600020178435018


In [21]:
sent_tokens = list()                            #здесь храним предложения
for i in range(len(sents)):
    sent_tokens.append(tknzr.tokenize(sents[i]))    

In [22]:
#сох нормальные формы слов
morph = pm2.MorphAnalyzer()     
vocab_normalized = sorted(set([morph.parse(w)[0].normalized for w in words]), key = lambda w: w.normal_form)   

In [ ]:
def progress(full, curr, checkpoint):
    if curr / full * 100 > checkpoint:
        print(curr / full * 100, end = '')
        print('% complited')
        return True
    
    else:
        return False
    
i = 0
h = 0.1
checkpoint = h
time = tm.clock()
for sent in sents:
    if progress(len(sents), i, checkpoint):
        time_per_checkpoint = tm.clock() - time
        print('Time per checkpoint:', end = ' ')
        print(time_per_checkpoint)
        print('Time per sentence:', end = ' ')
        time_per_sent = time_per_checkpoint / (len(sents) * h)
        print(time_per_sent)
        print('Estimated time:', end = ' ')
        print(time_per_checkpoint * (100 - checkpoint))
        time = tm.clock()
        checkpoint = checkpoint + h
        
    toks = tknzr.tokenize(sent)
    toks = [t.lower() for t in toks if t.isalnum()]
    sentence = ''
    for t in toks:
        sentence = sentence + t + ' '
    
    lemma_raw = isp_api.lemmatizationAnnotate(text = sentence)
    lemmatized = ''
    if lemma_raw.get('annotations').get('lemma') is not None:
        for l in lemma_raw.get('annotations').get('lemma'):
            lemmatized = lemmatized + l.get('value') + ' '
        
        sents[i] = lemmatized
    
    else:
        sents[i] = ''
        
    i = i + 1
        
    
sents[0]

In [2]:
with open('lemmatized.txt', 'w', encoding='utf8') as f:
    for s in sents[0:5000]:
        f.write(s + '\n')

NameError: name 'sents' is not defined

In [11]:
model = gensim.models.Word2Vec(gensim.models.word2vec.LineSentence('lemmatized.txt'), min_count=10)


In [93]:
model.most_similar(positive = ['айфон'])

[('а', 0.9998720288276672),
 ('они', 0.9998717904090881),
 ('где', 0.9998696446418762),
 ('этот', 0.9998689293861389),
 ('мочь', 0.9998672604560852),
 ('когда', 0.999864935874939),
 ('это', 0.9998648166656494),
 ('там', 0.9998632669448853),
 ('с', 0.9998587965965271),
 ('ничто', 0.9998581409454346)]

In [5]:
i = 5

message = 'сейчас я говорю о том человеке который спит отдыхая на работе'
message_tokens = tokenizer.tokenize(message)
parse_tokens = [morph.parse(t) for t in message_tokens]
#print(parse_tokens[i])
isp_POS = isp_api.posTaggingAnnotate(text = message)
isp_lemma = isp_api.lemmatizationAnnotate(text = message)

#print(isp_POS['annotations']['pos-token'][i])

#print(isp_lemma.get('annotations').get('lemma')[i].get('value'))

#print(' '.join([l.get('value') for l in isp_lemma.get('annotations').get('lemma')]))

for j in range(len(message_tokens)):
    isp_lemma_token = isp_lemma.get('annotations').get('lemma')[j].get('value')
    morph_parse_token = [w.normalized for w in morph.parse(isp_lemma_token) if w.normal_form == w.word]
    print(isp_lemma_token)
    print(isp_POS['annotations']['pos-token'][j])
    print(morph_parse_token)
    print("\n")

сейчас
{'value': {'characters': [], 'tag': 'ADV', 'type': 'syn-tag-rus'}, 'start': 0, 'text': 'сейчас', 'end': 6, 'annotated-text': 'сейчас я говорю о том человеке который спит отдыхая на работе'}
[Parse(word='сейчас', tag=OpencorporaTag('ADVB'), normal_form='сейчас', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сейчас', 3, 0),))]


я
{'value': {'characters': [{'tag': 'Masculine', 'type': 'gender'}, {'tag': 'DEICTIC', 'type': 'pronoun'}, {'tag': 'Singular', 'type': 'number'}, {'tag': 'Nominative', 'type': 'case'}, {'tag': 'Animated', 'type': 'animacy'}], 'tag': 'S', 'type': 'syn-tag-rus'}, 'start': 7, 'text': 'я', 'end': 8, 'annotated-text': 'сейчас я говорю о том человеке который спит отдыхая на работе'}
[Parse(word='я', tag=OpencorporaTag('NPRO,1per sing,nomn'), normal_form='я', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'я', 3100, 0),))]


говорить
{'value': {'characters': [{'tag': 'First', 'type': 'person'}, {'tag': 'Singular', 'type': 'number'}, {'tag': 'NotPast', 'type

In [76]:
ws = ['встать', 'вставать', 'встали', 'вставали']
for w in ws:
    print(w)
    ns = morph.parse(w)
    ts = isp_api.lemmatizationAnnotate(text = w)
    print(ts)
    print(ns)
        
    print('\n')

встать
{'text': 'встать', 'annotations': {'lemma': [{'text': 'встать', 'value': 'вставать', 'annotated-text': 'встать', 'start': 0, 'end': 6}]}}
[Parse(word='встать', tag=OpencorporaTag('INFN,perf,intr'), normal_form='встать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'встать', 904, 0),))]


вставать
{'text': 'вставать', 'annotations': {'lemma': [{'text': 'вставать', 'value': 'вставать', 'annotated-text': 'вставать', 'start': 0, 'end': 8}]}}
[Parse(word='вставать', tag=OpencorporaTag('INFN,impf,intr'), normal_form='вставать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'вставать', 903, 0),))]


встали
{'text': 'встали', 'annotations': {'lemma': [{'text': 'встали', 'value': 'вставать', 'annotated-text': 'встали', 'start': 0, 'end': 6}]}}
[Parse(word='встали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='встать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'встали', 904, 4),))]


вставали
{'text': 'вставали', 'annotations': {'lemma': [{'text': 'встав